In [5]:
import numpy as np
import cv2 as cv2
import datetime
from cv2 import aruco
import yaml

In [ ]:
# Define ARUCO dictionary to detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

#Initialize the video capture (1 for computer camera)
cap = cv2.VideoCapture(0)

# Load calibration data
with open('calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)
mtx = np.array(calibration_data['camera_matrix'])
dist = np.array(calibration_data['dist_coeff'])

# Get the optimal camera matrix
ret, img = cap.read()
if not ret:
    print("Error: Could not read from the camera.")
    exit()

h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


while True:
    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break

    # Convert frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Undistort the image
    dst = cv2.undistort(gray, mtx, dist, None, newcameramtx)


    
    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(dst, aruco_dict, parameters=parameters)


    
    if ids is not None and len(ids) >= 4:  # Ensure at least 4 markers are detected
        # Draw detected markers for visualization
        copy_frame = frame
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)
        


        # Collect the corners of the 4 markers
        marker_corners = {}
        for i, marker_id in enumerate(ids.flatten()):
            marker_corners[marker_id] = corners[i][0]  # Extract the corners for each detected marker

        # Check if specific 4 marker IDs are detected (e.g., IDs 0, 1, 2, 3)
        required_ids = [1, 2, 3, 4]
        if all(marker_id in marker_corners for marker_id in required_ids):
            # Get the centers of the 4 markers
            centers = {marker_id: np.mean(marker_corners[marker_id], axis=0) for marker_id in required_ids}

            # Sort the points for consistent perspective correctionqq
            # Top-left, top-right, bottom-right, bottom-left order
            sorted_ids = sorted(centers.items(), key=lambda x: (x[1][1], x[1][0]))  # Sort by Y, then by X
            sorted_points = [x[1] for x in sorted_ids]  # Extract the points in sorted order

            # Create source and destination points for perspective transform
            src_points = np.array(sorted_points, dtype="float32")
            a4_width = 248  # Adjust width in pixels
            a4_height = int(a4_width * np.sqrt(2))  # Calculate height to maintain A4 ratio

            #dst_points = np.array([[0, 0], [a4_width, 0], [a4_width, a4_height], [0, a4_height]], dtype="float32")
            
            marker_size_px = 50  # Approximate size of an ArUco marker in pixels
            crop_margin = marker_size_px // 4  # Define the inward crop margin as 1/4 of the marker size
            
            # Adjusted destination points with inward cropping
            dst_points = np.array([
                [crop_margin, crop_margin],                           # Top-left
                [a4_width - crop_margin, crop_margin],                # Top-right
                [a4_width - crop_margin, a4_height - crop_margin],    # Bottom-right
                [crop_margin, a4_height - crop_margin]                # Bottom-left
            ], dtype="float32")
             
            
            # Compute the perspective transform matrix
            matrix = cv2.getPerspectiveTransform(src_points, dst_points)

            # Warp the perspective to get the focused rectangle
            warped = cv2.warpPerspective(copy_frame, matrix, (a4_width, a4_height))

            # Display the warped image (focused rectangle)
            cv2.imshow("Warped View", warped)


            # Apply threshold to detect black regions (obstacles)
            _, binary_map = cv2.threshold(warped, 50, 255, cv2.THRESH_BINARY)

            # Display the obstacle map (1 for obstacle, 0 for no obstacle)
            cv2.imshow("Obstacle Map", binary_map)



            

    # Display the original frame with detected markers
    cv2.imshow("Original Frame", frame)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 ...

 [[255   0   0]
  [255   0   0]
  [255   0   0]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255   0   0]
  [255 255 255]
  [255 255 255]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]

 [[255   0   0]
  [255 255 255]
  [255   0   0]
  ...
  [255 255 255]
  [255 255 255]
  [255 255 255]]]
[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [  0   0   0]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [  0   0   0]
  [  0   0   0]
  [255   0   0]]

 ...

 [[255   0   0]
  [255   0   0]
  [255   0   0]
  ..

In [3]:
# Define ARUCO dictionary to detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

#Initialize the video capture (1 for computer camera)
cap = cv2.VideoCapture(0)

# Load calibration data
with open('calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)
mtx = np.array(calibration_data['camera_matrix'])
dist = np.array(calibration_data['dist_coeff'])

# Get the optimal camera matrix
ret, img = cap.read()
if not ret:
    print("Error: Could not read from the camera.")
    exit()

h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


while True:
    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break

    # Convert frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Undistort the image
    dst = cv2.undistort(gray, mtx, dist, None, newcameramtx)

    
    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(dst, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 4:  # Ensure at least 4 markers are detected
        # Draw detected markers for visualization
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        # Collect the corners of the 4 markers
        marker_corners = {}
        for i, marker_id in enumerate(ids.flatten()):
            marker_corners[marker_id] = corners[i][0]  # Extract the corners for each detected marker

        # Check if specific 4 marker IDs are detected (e.g., IDs 0, 1, 2, 3)
        required_ids = [1, 2, 3, 4]
        if all(marker_id in marker_corners for marker_id in required_ids):
        # Get the centers of the 4 markers
            centers = {marker_id: np.mean(marker_corners[marker_id], axis=0) for marker_id in required_ids}

            # Sort the points for consistent perspective correction
            # Top-left, top-right, bottom-right, bottom-left order
            sorted_ids = sorted(centers.items(), key=lambda x: (x[1][1], x[1][0]))  # Sort by Y, then by X
            sorted_points = [x[1] for x in sorted_ids]  # Extract the points in sorted order


            # Create source and destination points for perspective transform
            src_points = np.array(sorted_points, dtype="float32")

            # Define the desired output rectangle dimensions
            a4_width = 248  # Width in pixels
            a4_height = int(a4_width * np.sqrt(2))  # Height to maintain A4 aspect ratio
            
            # Define the destination points for perspective transformation
            dst_points = np.array([
                [0, 0],                  # Top-left
                [a4_width - 1, 0],       # Top-right
                [a4_width - 1, a4_height - 1],  # Bottom-right
                [0, a4_height - 1]       # Bottom-left
            ], dtype="float32")
                         
            
            # Compute the perspective transform matrix
            matrix = cv2.getPerspectiveTransform(src_points, dst_points)

            # Warp the perspective to get the focused rectangle
            warped = cv2.warpPerspective(frame, matrix, (a4_width, a4_height))

            # Display the warped image (focused rectangle)
            cv2.imshow("Warped View", warped)


            # Apply threshold to detect black regions (obstacles)
            _, binary_map = cv2.threshold(warped, 50, 255, cv2.THRESH_BINARY)


            # Display the obstacle map (1 for obstacle, 0 for no obstacle)
            cv2.imshow("Obstacle Map", binary_map)



            

    # Display the original frame with detected markers
    cv2.imshow("Original Frame", frame)


    

    



In [43]:
def image_smoothing(img, sc = 1000, ss = 3000, diameter = 20):
    smooth_img = cv2.bilateralFilter(img, d=diameter, sigmaColor = sc, sigmaSpace = ss)
    return smooth_img

def image_hsv(img):
    img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    return img_hsv

def get_obstacles(frame):
    img_copy = frame.copy()

    #1) Convert to HSV
    img_hsv = image_hsv(img_copy)

    #2) Smooth image to remove noise and smoothen the gradiant
    img_smooth = image_smoothing(img_hsv)

    #3) Mask for obstacles
    #obstaces colour (black). Colour red when detected
    obst_bound = np.array([[0, 0, 0], [180,220,90], [0, 0 , 255]])
    min_bound = np.array(obst_bound[0], np.uint8)
    max_bound = np.array(obst_bound[1], np.uint8)
    img_masked = cv2.inRange(img_smooth, min_bound, max_bound)
    

    #4) obstacle detection
    contours, _ = cv2.findContours(img_masked, cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    return contours


image_path = "output_frame.jpg" 

#cap = cv2.VideoCapture(0)
#ret, frame = cap.read()

frame = cv2.imread(image_path)
contours = get_obstacles(frame)
# Display the result
cv2.drawContours(frame, contours, -1, (0, 255, 0), 1)
cv2.imshow("Corrected Rectangle", frame)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [3]:
import cv2
import numpy as np
import yaml

# Define ARUCO dictionary to detect
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_6X6_250)
parameters = cv2.aruco.DetectorParameters()

# Initialize the video capture (1 for computer camera)
cap = cv2.VideoCapture(0)

# Load calibration data
with open('calibration.yaml', 'r') as f:
    calibration_data = yaml.safe_load(f)
mtx = np.array(calibration_data['camera_matrix'])
dist = np.array(calibration_data['dist_coeff'])

# Get the optimal camera matrix
ret, img = cap.read()
if not ret:
    print("Error: Could not read from the camera.")
    exit()

h, w = img.shape[:2]
newcameramtx, roi = cv2.getOptimalNewCameraMatrix(mtx, dist, (w, h), 1, (w, h))


while True:
    # Exit on pressing 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

    ret, frame = cap.read()
    if not ret:
        print("Error: Could not read frame")
        break

    # Convert frame to grayscale for detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Undistort the image
    dst = cv2.undistort(gray, mtx, dist, None, newcameramtx)
    
    # Detect ArUco markers
    corners, ids, rejected = cv2.aruco.detectMarkers(dst, aruco_dict, parameters=parameters)

    if ids is not None and len(ids) >= 6:  # Ensure at least 6 markers are detected
        # Draw detected markers for visualization
        cv2.aruco.drawDetectedMarkers(frame, corners, ids)

        # Collect the corners of the markers
        marker_corners = {}
        for i, marker_id in enumerate(ids.flatten()):
            marker_corners[marker_id] = corners[i][0]  # Extract the corners for each detected marker

        # Check if the required markers (IDs 1, 2, 3, 4, 5, 6) are detected
        required_ids = [1, 2, 3, 4, 5, 6]
        if all(marker_id in marker_corners for marker_id in required_ids):
            # Get the centers of the 4 markers for perspective correction (IDs 1, 2, 3, 4)
            centers = {marker_id: np.mean(marker_corners[marker_id], axis=0) for marker_id in required_ids[:4]}

            # Sort the points for consistent perspective correction
            # Top-left, top-right, bottom-right, bottom-left order
            sorted_ids = sorted(centers.items(), key=lambda x: (x[1][1], x[1][0]))  # Sort by Y, then by X
            sorted_points = [x[1] for x in sorted_ids]  # Extract the points in sorted order

            # Create source and destination points for perspective transform
            src_points = np.array(sorted_points, dtype="float32")
            a4_width = 248  # Adjust width in pixels
            a4_height = int(a4_width * np.sqrt(2))  # Calculate height to maintain A4 ratio

            dst_points = np.array([[0, 0], [a4_width, 0], [a4_width, a4_height], [0, a4_height]], dtype="float32")

            # Compute the perspective transform matrix
            matrix = cv2.getPerspectiveTransform(src_points, dst_points)

            # Warp the perspective to get the focused rectangle (A4 size)
            warped = cv2.warpPerspective(frame, matrix, (a4_width, a4_height))

            # Display the warped image (focused rectangle)
            cv2.imshow("Warped View", warped)

            # Convert the warped image to grayscale to detect obstacles
            warped_gray = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)

            # Apply threshold to detect black regions (obstacles)
            _, binary_map = cv2.threshold(warped_gray, 50, 255, cv2.THRESH_BINARY_INV)

            # Invert the binary map so that 0 represents obstacles and 1 represents free space
            obstacle_map = cv2.bitwise_not(binary_map)

            # Display the obstacle map (0 for obstacle, 1 for no obstacle)
            cv2.imshow("Obstacle Map", obstacle_map)

            # Get the start (ID 5) and end (ID 6) marker positions
            start_marker_corners = marker_corners[5]
            end_marker_corners = marker_corners[6]

            # Get the centers of the start and end markers
            start_center = np.mean(start_marker_corners, axis=0)
            end_center = np.mean(end_marker_corners, axis=0)

            # Transform the marker centers to the warped map using the perspective matrix
            start_point = cv2.perspectiveTransform(np.array([start_center], dtype="float32").reshape(-1, 1, 2), matrix)
            end_point = cv2.perspectiveTransform(np.array([end_center], dtype="float32").reshape(-1, 1, 2), matrix)

            # Draw the start and end points on the warped map
            start_point = start_point[0][0]
            end_point = end_point[0][0]

            # Draw a circle at the start point (green) and end point (red) on the warped map
            cv2.circle(warped, tuple(np.int32(start_point)), 10, (0, 255, 0), -1)  # Green for start
            cv2.circle(warped, tuple(np.int32(end_point)), 10, (0, 0, 255), -1)  # Red for end

            # Display the updated warped map with the start and end points
            cv2.imshow("Warped View with Start and End", warped)

    # Display the original frame with detected markers
    cv2.imshow("Original Frame", frame)

# Release resources
cap.release()
cv2.destroyAllWindows()
